In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve


In [ ]:
df = pd.read_csv("diabetes.csv")
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [ ]:
rf_model = RandomForestClassifier(random_state=17)

In [ ]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 22,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 28,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 17,
 'verbose': 0,
 'warm_start': False}

In [ ]:
rf_random_params = {"max_depth": np.random.randint(5,50,10),
                    "max_features":[3, 5, 7, "auto","sqrt"],
                    "min_samples_split": np.random.randint(2, 50, 20),
                    "n_estimators":[int(x) for x in np.linspace(start=200, stop=1500, num=10)]}

Hayat Kurtaran Serisinden !

Gidilecek yönü bimiyorsanız, ilgili algoritma içerisinde detaylı bir çalışma imkanınız olmadıysa öncelikle random search cv yardımıyla geniş bir hiper parametre seti girip, random search ile arama yapıp sonucunda optimum olarak bulduğunuz hiperparametrelerin etrafına yeni değerler koyarak gread search yapılması tavsiye edilir.

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf_model,
                               param_distributions = rf_random_params,
                               n_iter =100, #denenecek parametre sayısı
                               cv= 3,
                               verbose = True,
                               random_state = 42,
                               n_jobs=-1)

In [ ]:
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=17),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': array([22,  9,  5, 12, 47,  5, 42,  6, 14, 44]),
                                        'max_features': [3, 5, 7, 'auto',
                                                         'sqrt'],
                                        'min_samples_split': array([42, 34, 22, 34,  8, 26, 28, 37, 49,  6, 44, 45, 42, 39, 30, 49,  6,
       49, 33,  7]),
                                        'n_estimators': [200, 344, 488, 633,
                                                         777, 922, 1066, 1211,
                                                         1355, 1500]},
                   random_state=42, verbose=True)

In [ ]:
rf_random.best_params_

{'max_depth': 22,
 'max_features': 'sqrt',
 'min_samples_split': 28,
 'n_estimators': 200}

In [ ]:
rf_model_final = rf_model.set_params(**rf_random.best_params_, random_state = 17).fit(X,y)

In [ ]:
def cv_func_result(model, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"],returns=False):
  cv_results = cross_validate(model, X=X, y=y, cv=cv, scoring=scoring)
  for col in scoring:
    if col == "accuracy":
      print("Accuracy test mean = {0}".format(cv_results["test_accuracy"].mean()))
    elif col == "f1":
      print("F1 test mean = {0}".format(cv_results["test_f1"].mean()))
    elif col == "roc_auc":
      print("ROC_AUC test mean = {0}".format(cv_results["test_roc_auc"].mean()))
  if returns:
    return cv_results

cv_func_result(rf_model_final, X, y, cv=5)

Accuracy test mean = 0.7682879212290977
F1 test mean = 0.6293202548677237
ROC_AUC test mean = 0.8361495457721873


Bu bulduğumuz parametreler üzerinden gread search yapacağımız parametreler ekleyerek kendimiz deneyelim :

In [ ]:
rf_params = {"max_depth":[12, 22, 32],
              "max_features":[7, "auto","sqrt"],
              "min_samples_split": [16, 28, 35],
              "n_estimators":[200, 150,300]}

def model_best_params(model, params, cv=5, n_jobs=-1, verbose =True, random_state =17):
  model_best_grid = GridSearchCV(model, params, cv=cv, n_jobs = n_jobs, verbose = verbose).fit(X,y)
  for x in model_best_grid.best_params_.keys():
    print('For {0} best param {1}'.format(x, model_best_grid.best_params_[x]))
  
  print("Set with the best parameters of your model...")
  model_final = model.set_params(**model_best_grid.best_params_, random_state = random_state).fit(X,y)
  return model_final

rf_final = model_best_params(rf_model, rf_params)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
For max_depth best param 12
For max_features best param auto
For min_samples_split best param 16
For n_estimators best param 200
Set with the best parameters of your model...


Yeni hiperparametrelerimizden en iyileri ekranda gördüğünüz gibi çıktı. Bunun üzerinden bir de hatalarına bakalım bir önceki duruma göre fark var mı ?

In [ ]:
cv_func_result(rf_final, X, y, cv=5)

Accuracy test mean = 0.7722349545878957
F1 test mean = 0.6411009021749068
ROC_AUC test mean = 0.8365136268343816


f1 ve roc_auc da biraz artma olmuş ancak accuracy değerimiz düşmüş. Belki daha iyi aralıklar girseydim accuracy değerim de artardı. Denemekten zarar gelmez görmüş olduk.